In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import os
from dotenv import load_dotenv
from pyspark.sql import functions as sf

load_dotenv('../.env')

# Create a SparkSession
spark = SparkSession.builder \
    .appName("CSV to MySQL") \
    .config("spark.jars", "./mysql-connector-java-5.1.34_1.jar") \
    .getOrCreate()

23/07/02 17:34:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
mysql_properties = {
    "url": "jdbc:mysql://172.18.0.1:3306/acme",
    "driver": "com.mysql.jdbc.Driver",
    "user": "root",
    "password": "password",
    "dbtable": "badge"
}

In [47]:
df = spark.read.format("csv") \
    .option("header", "true") \
    .load("../employees - infos badge à mettre à jour.csv")

In [58]:
df = df.fillna('').withColumn('full_name', sf.concat(sf.col('employee_first_name'),sf.lit('_'), sf.col('employee_last_name')))
df

DataFrame[employee_first_name: string, employee_last_name: string, badge_serial_number: string, full_name: string]

In [60]:
query = "select id, first_name, lastName from employee where (first_name, lastName) \n" +\
    "IN ( SELECT first_name, lastName FROM employee GROUP BY first_name, lastName having count(*)=1)"
employee_df = spark.read \
                    .format("jdbc") \
                    .option("driver", "com.mysql.jdbc.Driver") \
                    .option("url", "jdbc:mysql://{}:{}/{}".format(os.getenv('mysql_hostname'),os.getenv('mysql_port'), os.getenv('mysql_db'))) \
                    .option("user", os.getenv('mysql_user')) \
                    .option("password", "password") \
                    .option("query", query)\
                    .load()

In [61]:
employee_df = employee_df.withColumn('full_name_e', sf.concat(sf.col('first_name'),sf.lit('_'), sf.col('lastName')))
employee_df

DataFrame[id: int, first_name: string, lastName: string, full_name_e: string]

In [64]:
x = employee_df.join(df, 
                     on=(employee_df['full_name_e']==df['full_name']),
                     how='inner')
len(x.collect())

1158

In [14]:
import pandas as pd

In [65]:
pd.set_option('display.max_columns', 10)
df_pd = x.select(sf.col('id'), sf.col('full_name'), sf.col('full_name_e'), sf.col('badge_serial_number')).toPandas()
pd.set_option('display.max_rows', df_pd.shape[0]+1)
df_pd

,id,full_name,full_name_e,badge_serial_number
0,1,Aldus_Kelly,Aldus_Kelly,4180485c-ad12-45d0-a4e9-99b0db7a5f75
1,2,Alford_Cooper,Alford_Cooper,d7cfd37c-bddf-41a3-9f2b-390698f443cb
2,3,Audrey_Montgomery,Audrey_Montgomery,3e4c228f-5a7e-4bad-8d09-773dbe3cce4e
3,4,Byron_Jones,Byron_Jones,1867811c-5564-41e8-803a-edcac63c38bb
4,5,Reid_Chapman,Reid_Chapman,ddaac01a-7cbd-41bf-8448-34f6196acf2f
5,6,Rebecca_Adams,Rebecca_Adams,f631fbc4-35f0-4672-9df6-156423bf70d0
6,7,Sabrina_Mitchell,Sabrina_Mitchell,e9641a57-3e2e-4b3d-b20f-8403046e6422
7,8,Vanessa_Clark,Vanessa_Clark,4c8c22d2-6544-4f2a-b301-cf27fa87cf16
8,9,Alissa_Clark,Alissa_Clark,199df8a7-a262-49bc-bfe6-d97072df5be9
9,10,Dominik_Robinson,Dominik_Robinson,a0f8c733-d0f3-4b1a-ab4b-f78151d0cdeb
